# Now we will build a simple UI to interact with our model

Let's create our first Streamlit app

In [ ]:
%%writefile streamlit_app.py

import streamlit as st

st.title('Interact with Finnish GPT-3')

# Creating a text input box
text_input = st.text_input("Please enter your prompt:")

# Creating a submit button
if st.button("Submit"):
    # Display the hello message
    st.write(f"Hello {text_input}!")

Now, you open a terminal (File -> New -> Terminal) and run:

1. `pip install streamlit`
2. `streamlit run streamlit_app.py --server.port 6006`

Next, open the following page (run the following cell to generate the hyperlink):

In [ ]:
import json
import boto3

region = boto3.session.Session().region_name
with open("/opt/ml/metadata/resource-metadata.json", 'r') as file_:
    domainId = json.loads(file_.read())['DomainId']

print(f"https://{domainId}.studio.{region}.sagemaker.aws/jupyter/default/proxy/6006/")



## Keep making changes below, and they will be reflected in the website above!

In [ ]:
%%writefile streamlit_app.py

import streamlit as st
import sagemaker
import json

sm_endpoint_name = "TurkuNLP-gpt3-finnish-3B"
predictor = sagemaker.Predictor(endpoint_name=sm_endpoint_name)

@st.cache_data
def invoke_model(text, temperature, length):
    serde = {
        "ContentType": "application/json",
        "Accept": "application/json",
    }
    
    payload = {
        "inputs": text,
        "parameters": {
            "temperature": float(temperature),
            "max_new_tokens": int(length)
        }
    }
    payload = json.dumps(payload).encode('utf-8')

    result = predictor.predict(payload, serde)
    
    st.write(f"{json.loads(result)[0]['generated_text']}!")

st.title('Interact with Finnish GPT-3')

# Creating a text input box
text_input = st.text_area("Please enter your prompt:", placeholder="Hi, this is a sample prompt")

temperature = st.sidebar.slider('Temperature', 0.1, 1.0, 0.7)

length = st.sidebar.slider('Max tokens', 0, 500, 50)

# Creating a submit button
if st.button("Submit"):

    invoke_model(text_input, temperature, length)
    
